# Data Integration
 Purpose: This Notebook Takes all the matched entities from Drugs, Manufacturer, and Physician entity resolutions and creates the tables to be loaded into Neo4j
 
## Importing dependencies

In [343]:
import pandas as pd
import numpy as np
import pickle

## Importing Entity Resolution Lookup tables
### Manufacturer Node

In [484]:
# manuf_lookup = pd.read_csv('../Data/Outputs_Cleanup/Manufacturer_entity_matching/Manufacturer_Matching_lookup.csv')
manuf_lookup = pickle.load(open('../Data/Outputs_Cleanup/Manufacturer_entity_matching/Manufacturer_Matching_lookup.p','rb'))
manuf_lookup[['manuf_id','manufacturer_name']].to_csv('../Nodes/Manufacturer.csv')
manuf_lookup.explode('fda_manuf_id')

,manuf_id,manufacturer_name,fda_manuf_id,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID
0,0,LANTHEUS MEDICAL IMAGING INC,20843,100000005400.0
1,1,SEBELA PHARMACEUTICALS INC,2940,100000066371.0
2,2,BRAINTREE LABORATORIES INC,31992,100000000153.0
3,3,BIODELIVERY SCIENCES INTERNATIONAL INC,18033,100000061387.0
4,4,CHIESI USA INC,212,100000010860.0
...,...,...,...,...
10642,10642,LABORIE MEDICAL TECHNOLOGIES CORP,NaN,100000136464.0
10643,10643,ERBE ELEKTROMEDIZIN GMBH,NaN,100000806830.0
10644,10644,IPSEN PHARMA SAS,NaN,100000010667.0
10645,10645,MEDTEC INC,NaN,100000011192.0


### Drug Node

In [485]:
def string_to_ordered_list (x):
    '''
    Function that takes a string that looks like a list, and converts it to an ordered list
    '''
    ordered_list = None
    try: 
        str_list = str(x).replace('[','').replace(']','').replace(' ','').replace("'",'').split(',')
        str_list = list(map(int,str_list))
        str_set = set(str_list)
        ordered_list = sorted(list(str_set))
    except:
        ordered_list = np.NaN
    
    
    return ordered_list

In [486]:
## Importing drug matching output
drug_node = pd.read_csv('../Data/Outputs_Cleanup/Sunshine_dedupe_drug_integration/final_drug_lookup.csv')
## Importing drug information from the deduped FDA data
openfda_drug_dedupe = pickle.load(open( "../Data/Outputs_Cleanup/FDA/Openfda_Drug_deduplicated.p", "rb" ))

## Sorting the fda_drug_id list column so that when we convert to a string, it will 
## match the fda_drug_id list column in the drug_node dataframe
openfda_drug_dedupe = openfda_drug_dedupe.applymap(set).applymap(list)
openfda_drug_dedupe.fda_drug_id = openfda_drug_dedupe.fda_drug_id.apply(sorted)

## converting the fda_drug_id string column into a sorted list column
drug_node['fda_drug_id'] = drug_node.fda_drug_id.apply(string_to_ordered_list)

## converting fda_drug_id columns in drug_node df and openfda_drug_dedupe df 
## to string from list for merging
drug_node['fda_drug_id'] = drug_node['fda_drug_id'].astype(str)
openfda_drug_dedupe['fda_drug_id'] = openfda_drug_dedupe['fda_drug_id'].astype(str)
drug_node = drug_node.merge(openfda_drug_dedupe,'left','fda_drug_id')

## Dropping id's into other datasources
drug_node_output = drug_node.drop(columns = ['fda_drug_id','MedD_drug_Id','sunshine_drug_id','manufacturer_name'])
drug_node_output.to_csv('../Nodes/Drug_Node.csv')

## Starting with the Sunshine Data
### Hospital Node

In [430]:
hospital_node = pd.read_csv('../Data/Outputs_Cleanup/Sunshine/hosp_info.csv')
hospital_node.to_csv('../Nodes/Hospital_Node.csv')

### Payments Node

In [489]:
payments_node = pd.read_csv('../Data/Outputs_Cleanup/Sunshine/payments.csv')
payments_node.to_csv('../Nodes/Payments_Node.csv')
# payments_node

### Manufacturer-Payments Edge

In [432]:
manuf_payments_edge = pd.read_csv('../Data/Outputs_Cleanup/Sunshine/pharmCo_payments.csv')
manuf_payments_edge = manuf_payments_edge.merge(manuf_lookup,'left','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID')
manuf_payments_edge = manuf_payments_edge[['Record_ID','manuf_id']]
manuf_payments_edge.to_csv('../Edges/Manufacturer_Payment_Edge.csv')

### Payment-Drug Edge

In [478]:
payment_drug_edge = pd.read_csv('../Data/Outputs_Cleanup/Sunshine/record_drugs.csv')
cols = ['Record_ID','Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1',
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2',
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4',
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5'
       ]
payment_drug_edge = payment_drug_edge[cols].melt(id_vars = 'Record_ID')
payment_drug_edge.dropna(inplace=True)
payment_drug_edge = payment_drug_edge[['Record_ID','value']]
payment_drug_edge.columns = ['Record_ID','sunshine_drug_id']
drug_node[['drug_id','sunshine_drug_id']].dropna()
payment_drug_edge = payment_drug_edge.merge(drug_node[['drug_id','sunshine_drug_id']].dropna(),
                                           'left',
                                           'sunshine_drug_id')
payment_drug_edge = payment_drug_edge[~payment_drug_edge.drug_id.isna()][['Record_ID','drug_id']]

payment_drug_edge.drug_id= payment_drug_edge.drug_id.astype(int)
payment_drug_edge.to_csv('../Edges/Payment_Drug_Edge.csv')

### Manufacturer-Drug edge

In [454]:
openfda_manuf_dedupe = pickle.load(open( "../Data/Outputs_Cleanup/FDA/openfda_manufacturer_deduplicated_single_manuf.p", "rb" ))

manuf_drug_edge = drug_node[['drug_id','manufacturer_name']]
manuf_drug_edge = manuf_drug_edge.explode('manufacturer_name')
manuf_drug_edge.dropna(inplace=True)
manuf_drug_edge = manuf_drug_edge.merge(openfda_manuf_dedupe.explode('manuf_names'),'left',left_on='manufacturer_name',right_on='manuf_names')
# manuf_drug_edge
# manuf_lookup
manuf_drug_edge = manuf_drug_edge[['drug_id','fda_manuf_id','manufacturer_name_y']].explode('fda_manuf_id')
manuf_drug_edge = manuf_drug_edge.drop_duplicates(['drug_id','manufacturer_name_y'])[['drug_id','fda_manuf_id']]
manuf_drug_edge['fda_manuf_id'] = manuf_drug_edge['fda_manuf_id'].astype(str)
manuf_drug_edge = manuf_drug_edge.merge(manuf_lookup.explode('fda_manuf_id'),'left','fda_manuf_id')
manuf_drug_edge = manuf_drug_edge[['drug_id','manuf_id']]
manuf_drug_edge.to_csv('../Edges/Manufacturer_Drug_Edge.csv')

### Prescription Node